In [46]:
#this is a notebook for running a few scripts to get the 
#original sales data from a csv format into pandas,
#as well as the relevant GIS building footprint into a csv file.
#This is more of a grab-bag of commands to get ready for some 
#pandas joins and a sql join (which I may execute from python)

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from geopy.geocoders import Nominatim
import datetime




In [47]:
#try the shpfile version
import pandashp as pdshp
df = pdshp.read_shp('data/BuildingFootprints/geo_export_3812582d-7726-4629-a875-09f9935aed3c')

In [48]:
#select only buildings via feat_code 2100, which is a building footprint
print(df.columns)
newpd = df[df['feat_code']==2100]
nrecs = newpd.shape[0]
del df

Index([u'num_floors', u'feat_code', u'groundelev', u'cnstrct_yr', u'name',
       u'doitt_id', u'date_lstmo', u'time_lstmo', u'bbl', u'bin',
       u'lststatype', u'shape_area', u'heightroof', u'built_code',
       u'shape_len', u'geometry'],
      dtype='object')


In [49]:
#calculate center of building and assign it to a new column
posArray = np.zeros((nrecs,2))
crdind = 0
for i in newpd.index:
    tempcrd = newpd.loc[i,'geometry'].centroid.coords[:]
    posArray[crdind,:] = tempcrd[0]
    crdind +=1 

In [55]:
# join it to a set of selected entries from new pd, then export to a csv
importantpd = newpd.loc[:,['doitt_id','bbl','bin']]
#add coordinates to frame
importantpd['long'] = pd.Series(posArray[:,0],index=importantpd.index)
importantpd['lat'] = pd.Series(posArray[:,1],index=importantpd.index)

#convert bbl in important pd to string, and NAN to 00000000000
naninds = np.isnan(importantpd['bbl'])
importantpd.loc[naninds,'bbl'] = ['00000000000' for item in importantpd.loc[naninds,'bbl']]
importantpd.loc[~naninds,'bbl'] = [str(int(item)) for item in importantpd.loc[~naninds,'bbl']]
#save this database as a csv for sql use
importantpd.to_csv('data/BuildingFootprints_augmented.csv')

In [51]:
#load a csv file of interest
bk15 = pd.read_csv('data/2015_brooklyn.csv',sep = ',')
#calculate the bbl and format it as string
borough = [str(item) for item in np.array(bk15.loc[:,'BOROUGH']).astype(int)]
block= [str(item).zfill(5) for item in np.array(bk15.loc[:,'BLOCK']).astype(int)]
lot = [str(item).zfill(4) for item in np.array(bk15.loc[:,'LOT']).astype(int)]
bbl = []
for k in range(len(borough)):
    bbl.append(borough[k]+block[k]+lot[k])
    
bk15['bbl'] = pd.Series(bbl,index=bk15.index)

In [52]:
#join the databases
joinedpd = pd.merge(bk15,importantpd,on='bbl')

In [53]:
joinedpd

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,...,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE,bbl,doitt_id,cnstrct_yr,bin,long,lat
0,3.0,BATH BEACH,01 ONE FAMILY DWELLINGS,1.0,6361.0,11.0,,A5,71 BAY 10TH STREET,,...,1.0,A5,829000.0,2016-04-05,3063610011,498448.0,1930.0,3166217.0,-74.010010,40.609505
1,3.0,BATH BEACH,01 ONE FAMILY DWELLINGS,1.0,6371.0,60.0,,A9,8620 19TH AVENUE,,...,1.0,A9,0.0,2015-09-16,3063710060,771618.0,1930.0,3166707.0,-74.001134,40.605598
2,3.0,BATH BEACH,01 ONE FAMILY DWELLINGS,1.0,6371.0,79.0,,A5,8668 19TH AVENUE,,...,1.0,A5,788000.0,2016-03-31,3063710079,627172.0,1930.0,3166718.0,-74.002024,40.604738
3,3.0,BATH BEACH,01 ONE FAMILY DWELLINGS,1.0,6373.0,73.0,,A1,50 BAY 23RD STREET,,...,1.0,A1,750000.0,2015-12-17,3063730073,411106.0,1930.0,3166829.0,-74.000249,40.604135
4,3.0,BATH BEACH,01 ONE FAMILY DWELLINGS,1.0,6394.0,5.0,,A5,160 BAY 10TH STREET,,...,1.0,A5,920000.0,2016-02-04,3063940005,225154.0,1940.0,3167338.0,-74.012188,40.607966
5,3.0,BATH BEACH,01 ONE FAMILY DWELLINGS,1.0,6398.0,39.0,,A1,1638 BENSON AVENUE,,...,1.0,A1,820000.0,2015-07-24,3063980039,594134.0,1925.0,3167572.0,-74.008041,40.607503
6,3.0,BATH BEACH,01 ONE FAMILY DWELLINGS,1.0,6401.0,7.0,,S1,1731 BATH AVENUE,,...,1.0,S1,40000.0,2016-02-10,3064010007,568112.0,1920.0,3167677.0,-74.007212,40.604731
7,3.0,BATH BEACH,01 ONE FAMILY DWELLINGS,1.0,6401.0,7.0,,S1,1731 BATH AVENUE,,...,1.0,S1,0.0,2015-10-17,3064010007,568112.0,1920.0,3167677.0,-74.007212,40.604731
8,3.0,BATH BEACH,01 ONE FAMILY DWELLINGS,1.0,6401.0,8.0,,S1,1729 BATH AVENUE,,...,1.0,S1,0.0,2016-02-07,3064010008,304572.0,1920.0,3167678.0,-74.007267,40.604764
9,3.0,BATH BEACH,01 ONE FAMILY DWELLINGS,1.0,6401.0,19.0,,A5,8733 BAY 16TH STREET,,...,1.0,A5,854000.0,2016-02-10,3064010019,703537.0,1925.0,3167685.0,-74.006775,40.605304


In [54]:
#write to csv
joinedpd.to_csv('data/bk15_augmented.csv')